# Converting nd2 files to .mat files readable by SMALL-LABS with 2x downsampling
For reading nd2 files collected with 40-ms integration time, for example, summing consecutive frames, and saving as .mat files, so that the effective integration time is doubled (~80 ms).
* working with hdf5storage 0.1.16 with Python 3.8.20
* environment called "smalllabs_mat" on Dan's desktop

## imports

In [ ]:
from glob import glob
import os
from os.path import join
from pathlib import Path
import shutil

import hdf5storage # version that works with SMALL-LABS is 0.1.16
import nd2
import numpy as np

## converting movies to .mat files

In [2]:
movie_folders = [
    r'T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\Wt Swi6 movies',
]

downsampled_folders = [
    r'T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\Wt Swi6 movies\2x downsample (80-ms)',
]

movie_pattern = '*.nd2'
downsample_appendix = '_80ms' # added to end of original movie name

for movie_folder, downsampled_folder in zip(movie_folders, downsampled_folders):
    movie_files = glob(join(movie_folder, movie_pattern))
    os.makedirs(downsampled_folder, exist_ok=True)

    for movie_file in movie_files:

        movie = nd2.imread(movie_file)

        num_frames_downsampled = movie.shape[0] // 2
        movie_downsampled = movie[0:num_frames_downsampled*2:2] + movie[1::2]

        movie_stem = Path(movie_file).stem
        
        data_dict = {}
        data_dict['mov'] = np.moveaxis(movie_downsampled,
                                       source = [0, 1, 2],
                                       destination = [2, 0, 1])

        hdf5storage.savemat(file_name = join(downsampled_folder, f"{movie_stem}{downsample_appendix}.mat"),
                            mdict = data_dict,
                            format = '7.3')

## copying over corresponding mask files

In [3]:
mask_folders = [
    r"T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\241017_Swi6_wt_reps_1_2",
    r"T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\241021_Swi6_wt_reps_3_4",
    r"T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\241023_Swi6_wt_rep_5"
]

downsampled_folders = [
    r'T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\Wt Swi6 movies\2x downsample (80-ms)',
    r'T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\Wt Swi6 movies\2x downsample (80-ms)',
    r'T:\MIGRATED\Lab_Members\Lexie_Moran\Data\202410\Wt Swi6 movies\2x downsample (80-ms)',
]

mask_pattern = "*_PhaseMask.mat"
downsample_appendix = '_80ms'

for mask_folder, downsampled_folder in zip(mask_folders, downsampled_folders):
    mask_files = glob(join(mask_folder, mask_pattern))
    os.makedirs(downsampled_folder, exist_ok=True)

    for mask_file in mask_files:
        mask_stem = Path(mask_file).stem
        insert_position = mask_stem.find("_PhaseMask")
        if insert_position != -1:
            downsampled_mask_stem = mask_stem[:insert_position] + downsample_appendix + mask_stem[insert_position:]
        destintation_file_name = join(downsampled_folder, f"{downsampled_mask_stem}.mat")
        shutil.copy(mask_file, destintation_file_name)